# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('eo_time.npz') 

# Access the concatenated features from the .npz file
X_time = data['X_time']

# Close the loaded file
data.close()

#print the merged array
X_time

array([[[-1.70983456e-06, -1.25198812e-06,  3.26074783e-11, ...,
          3.26074783e-11,  4.63140062e-01,  1.63413091e+00],
        [-2.32233246e-06, -2.01538933e-06,  2.75444251e-11, ...,
          2.75444251e-11,  4.68105779e-01,  1.54852559e+00],
        [-3.30576161e-08,  5.46407307e-07,  2.74209376e-11, ...,
          2.74209376e-11,  3.65731034e-01,  1.72778820e+00],
        ...,
        [-1.29114080e-07,  6.10958210e-07,  2.00323483e-11, ...,
          2.00323483e-11,  3.09365829e-01,  2.04954932e+00],
        [ 6.43081463e-07,  6.81324963e-07,  1.35657944e-11, ...,
          1.35657944e-11,  3.85630195e-01,  1.60045474e+00],
        [ 6.53286068e-07,  1.06357209e-06,  4.32161140e-11, ...,
          4.32161140e-11,  3.63171004e-01,  1.89534486e+00]],

       [[ 6.68200564e-07,  9.10121599e-07,  2.38972234e-11, ...,
          2.38972234e-11,  5.33616949e-01,  1.49649669e+00],
        [ 1.95829863e-06,  1.32217696e-06,  2.46533063e-11, ...,
          2.46533063e-11,  5.29122302e

In [3]:
data = np.load('eyes_open_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('eyes_open_groups.npz')

group = data['group']

data.close() 

group

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_time.shape, Y.shape, groups_array.shape)

(18145, 20, 13) (18145,) (18145,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256)))
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_time, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_time[train_index], X_time[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
64/64 [==============================] - 12s 51ms/step - loss: 0.3461 - accuracy: 0.8566 - val_loss: 0.3937 - val_accuracy: 0.9157
Epoch 2/200
64/64 [==============================] - 2s 27ms/step - loss: 0.1926 - accuracy: 0.9235 - val_loss: 0.2093 - val_accuracy: 0.9477
Epoch 3/200
64/64 [==============================] - 2s 27ms/step - loss: 0.1531 - accuracy: 0.9415 - val_loss: 0.1246 - val_accuracy: 0.9620
Epoch 4/200
64/64 [==============================] - 2s 27ms/step - loss: 0.1233 - accuracy: 0.9520 - val_loss: 0.0907 - val_accuracy: 0.9658
Epoch 5/200
64/64 [==============================] - 2s 27ms/step - loss: 0.0948 - accuracy: 0.9659 - val_loss: 0.0668 - val_accuracy: 0.9758
Epoch 6/200
64/64 [==============================] - 2s 25ms/step - loss: 0.0857 - accuracy: 0.9700 - val_loss: 0.0726 - val_accuracy: 0.9702
Epoch 7/200
64/64 [==============================] - 2s 28ms/step - loss: 0.0753 - accuracy: 0.9715 - val_loss: 0.0465 - val_accuracy: 0.982

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9972,0.9979,0.9968,0.9974,0.9973,0.9977,0.9968,0.9945,0.9945
1,0.9972,0.9979,0.9968,0.9974,0.9973,0.9977,0.9968,0.9945,0.9945
2,0.9978,0.9968,0.9989,0.9979,0.9977,0.9965,0.9989,0.9956,0.9956
3,0.9961,0.9937,0.9989,0.9963,0.9960,0.9931,0.9989,0.9923,0.9923
4,0.9983,0.9979,0.9989,0.9984,0.9983,0.9977,0.9989,0.9967,0.9967
5,0.9967,0.9979,0.9958,0.9968,0.9967,0.9977,0.9958,0.9934,0.9934
6,0.9983,0.9989,0.9979,0.9984,0.9984,0.9988,0.9979,0.9967,0.9967
7,0.9950,0.9958,0.9947,0.9952,0.9951,0.9954,0.9947,0.9901,0.9901
8,0.9939,0.9916,0.9968,0.9942,0.9938,0.9908,0.9968,0.9878,0.9879
9,0.9961,0.9979,0.9947,0.9963,0.9962,0.9977,0.9947,0.9923,0.9923


In [13]:
metrics_df.round(4).to_csv('TIME_EO_CNN_LSTM.csv', index = False)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
result = pd.read_csv("TIME_EO_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9972,0.9979,0.9968,0.9974,0.9973,0.9977,0.9968,0.9945,0.9945
1,0.9972,0.9979,0.9968,0.9974,0.9973,0.9977,0.9968,0.9945,0.9945
2,0.9978,0.9968,0.9989,0.9979,0.9977,0.9965,0.9989,0.9956,0.9956
3,0.9961,0.9937,0.9989,0.9963,0.9960,0.9931,0.9989,0.9923,0.9923
4,0.9983,0.9979,0.9989,0.9984,0.9983,0.9977,0.9989,0.9967,0.9967
5,0.9967,0.9979,0.9958,0.9968,0.9967,0.9977,0.9958,0.9934,0.9934
6,0.9983,0.9989,0.9979,0.9984,0.9984,0.9988,0.9979,0.9967,0.9967
7,0.9950,0.9958,0.9947,0.9952,0.9951,0.9954,0.9947,0.9901,0.9901
8,0.9939,0.9916,0.9968,0.9942,0.9938,0.9908,0.9968,0.9878,0.9879
9,0.9961,0.9979,0.9947,0.9963,0.9962,0.9977,0.9947,0.9923,0.9923


In [3]:
(result.mean()*100).round(2)

Accuracy       99.67
Precision      99.66
Recall         99.70
F1             99.68
Sensitivity    99.67
Specificity    99.63
ROC_AUC        99.70
Kappa          99.34
MCC            99.34
dtype: float64

In [18]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])